Deploy the model as a real time service that can be called via Azure ML SDK or REST request to make a prediction on new data.
To this end, deploy the model as a web service hosted in a container.

The model will be deployed as an Azure Container Instance (ACI) service that requires no authentication. This is fine for development and testing, but for production you should consider deploying to an Azure Kubernetes Service (AKS) cluster and enabling authentication. This would require REST requests to include an Authorization header.

# Import Libraries

In [1]:
# Standard libraries
import os

# Azure ML libraries
import azureml.core
from azureml.core import Model, Workspace
from azureml.core.conda_dependencies import CondaDependencies 
from azureml.core.model import InferenceConfig
from azureml.core.image import ContainerImage
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.webservice import Webservice, AksWebservice

# Azure ML Setup

In [2]:
# Load the workspace from the saved config file
ws = Workspace.from_config()
print("Ready to use Azure ML {} to work with {}".format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.0.83 to work with sbirkamlws


# List Models from Workspace

In [3]:
for model in Model.list(ws):
    print(model.name, "version:", model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ("\t",tag_name, ":", tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ("\t",prop_name, ":", prop)
    print("\n")

customer_churn_log_reg_model version: 4
	 Training Context : Estimator
	 Algorithm : Logistic Regression
	 AUC : 0.6876066509909305
	 Accuracy : 0.7889647326507395
	 Precision : 0.829666430092264
	 Recall : 0.8992307692307693
	 F Score : 0.8630490956072352


customer_churn_dt_model version: 3
	 Training Context : Notebook
	 Algorithm : Decision Tree


customer_churn_log_reg_model version: 3
	 Training Context : Estimator
	 Algorithm : Logistic Regression
	 AUC : 0.6865149479341619
	 Accuracy : 0.78839590443686
	 Precision : 0.8290780141843972
	 Recall : 0.8992307692307693
	 F Score : 0.862730627306273


customer_churn_dt_model version: 2
	 Training Context : Notebook
	 Algorithm : Decision Tree


customer_churn_log_reg_model version: 2
	 Training Context : Estimator
	 Algorithm : Logistic Regression
	 AUC : 0.6865149479341619
	 Accuracy : 0.78839590443686
	 Precision : 0.8290780141843972
	 Recall : 0.8992307692307693
	 F Score : 0.862730627306273


AutoMLd0a94946323 version: 1


custom

# Return a Model for Deployment

In [4]:
model = ws.models["customer_churn_log_reg_model"] # by default latest version
print(model.name, "version", model.version)

customer_churn_log_reg_model version 4


# Prepare Folder and Score File to Host Web Service for Deployment

In [5]:
folder_name = "customer_churn_service"

# Create a folder for the web service files
experiment_folder = './' + folder_name
os.makedirs(folder_name, exist_ok=True)

print(folder_name, "folder created.")

customer_churn_service folder created.


In [6]:
%%writefile $folder_name/score.py

import json
import joblib
import numpy as np
from azureml.core.model import Model
from azureml.core import Workspace
from sklearn.linear_model import LogisticRegression
import os

# Called when the service is loaded
def init():
    global model
    # Get the path to the deployed model file and load it
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'customer_churn_log_reg_model.pkl')
    model = joblib.load(model_path)

# Called when a request is received
def run(raw_data):
    # Get the input data as a numpy array
    data = np.array(json.loads(raw_data)["data"])
    # Get a prediction from the model
    predictions = model.predict(data)
    # Get the corresponding classname for each prediction (0 or 1)
    classnames = ["Non Churn", "Churn"]
    predicted_classes = []
    for prediction in predictions:
        predicted_classes.append(classnames[prediction])
    # Return the predictions as JSON
    return json.dumps(predicted_classes)

Overwriting customer_churn_service/score.py


# Create Dependencies and Inference Config

In [7]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies 

conda_deps = CondaDependencies.create(conda_packages=['numpy','scikit-learn==0.22.1'],
                                      pip_packages=['azureml-defaults','azureml-core'])
myenv = Environment(name='myenv')
myenv.python.conda_dependencies = conda_deps

# use an image available in public Container Registry without authentication
# myenv.docker.base_image = "mcr.microsoft.com/azureml/o16n-sample-user-base/ubuntu-miniconda"
# myenv.inferencing_stack_version='latest'

In [8]:
from azureml.core.model import InferenceConfig

inf_config = InferenceConfig(entry_script=os.path.join(folder_name,'score.py'), environment=myenv)

# Provision Azure Kubernetes Service

In [9]:
# Use the default configuration (can also provide parameters to customize)
prov_config = AksCompute.provisioning_configuration()

aks_name = 'sbirkamlaks' 

# Create the cluster if not already created
try:
    aks_target = ComputeTarget.create(workspace = ws, 
                                  name = aks_name, 
                                  provisioning_configuration = prov_config)
except:
    aks_target = ComputeTarget(workspace = ws, name = aks_name)

In [10]:
# Set the web service configuration (using default here)
aks_config = AksWebservice.deploy_configuration()

# # Enable token auth and disable (key) auth on the webservice
# aks_config = AksWebservice.deploy_configuration(token_auth_enabled=True, auth_enabled=False)

# Deploy Model

In [13]:
aks_service_name = "aks-service-3"

aks_service = Model.deploy(workspace=ws,
                           name=aks_service_name,
                           models=[model],
                           inference_config=inf_config,
                           deployment_config=aks_config,
                           deployment_target=aks_target)

aks_service.wait_for_deployment(show_output = True)
print(aks_service.state)

Running..
Succeeded
AKS service creation operation finished, operation "Succeeded"
Healthy


In [ ]:
aks_service.get_logs()

In [ ]:
'''# Add the dependencies for our model (AzureML defaults is already included)
myenv = CondaDependencies()
myenv.add_conda_package("scikit-learn==0.22.1")

# Save the environment config as a .yml file
env_file = folder_name + "/customer_churn_env.yml"
with open(env_file,"w") as f:
    f.write(myenv.serialize_to_string())
print("Saved dependency info in", env_file)

# Print the .yml file
with open(env_file,"r") as f:
    print(f.read())'''

In [ ]:
# Now you're ready to deploy. We'll deploy the container a service named customer-churn-service.
# The deployment process includes the following steps:

# 1. Define an inference configuration, which includes the scoring and environment files required to load and use the model.
# 2. Define a deployment configuration that defines the execution environment in which the service will be hosted.
#    In this case, an Azure Container Instance.
# 3. Deploy the model as a web service.
# 4. Verify the status of the deployed service.

# Deployment will take some time as it first runs a process to create a container image, and then runs a process 
# to create a web service based on the image. When deployment has completed successfully, you'll see a status of Healthy.

# Deploy Web Service in ACI

In [ ]:
''''# Configure the scoring environment
inference_config = InferenceConfig(runtime= "python",
                                   source_directory = folder_name,
                                   entry_script="score_customer_churn.py",
                                   conda_file="customer_churn_env.yml")

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)''''

In [ ]:
'''# Deploy model
service_name = "customer-churn-service-sdk"

service = Model.deploy(ws, service_name, [model], inference_config, deployment_config)

service.wait_for_deployment(True)
print(service.state)'''

In [ ]:
'''print(service.state)
print(service.get_logs())

# If you need to make a change and redeploy, you may need to delete unhealthy service using the following code:
#service.delete()'''

# List Web Services from Workspace

In [14]:
for webservice_name in ws.webservices:
    print(webservice_name)

aks-service-3
aks-service-1
aks-service-2
